# Differential Inverse Calculator

In [ ]:
import time

import numpy as np
from pydrake.all import (AddMultibodyPlantSceneGraph, BsplineTrajectory,
                         DiagramBuilder, KinematicTrajectoryOptimization,
                         MeshcatVisualizer, MeshcatVisualizerParams,
                         MinimumDistanceConstraint, Parser, PositionConstraint,
                         Rgba, RigidTransform, Role, Solve, Sphere,
                         StartMeshcat, JacobianWrtVariable, RollPitchYaw,
                         JointSliders, RotationMatrix,
                         InverseKinematics,
                         LeafSystem, AbstractValue,
                         Simulator, FixedOffsetFrame
                        )

from manipulation import running_as_notebook
from manipulation.meshcat_utils import (PublishPositionTrajectory,
                                        MeshcatPoseSliders,
                                        WsgButton)
from manipulation.scenarios import AddIiwaDifferentialIK, MakeManipulationStation, AddIiwa, AddPlanarIiwa, AddShape, AddWsg, AddMultibodyTriad 
from manipulation.utils import AddPackagePaths, FindResource

from IPython.display import clear_output

from utils import *


In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://68a898be-bd4e-4046-916a-6293aadf6c5d.deepnoteproject.com/7000/


To visualize throw and catch points

In [ ]:
squash_ball_radius = 0.02
p_GB_G = [0, 0.07, 0]
def add_target_objects(X_WThrow, X_WCatch):
    p_ThrowB_W = X_WThrow.rotation() @ p_GB_G
    X_WThrowB_W = RigidTransform(X_WThrow.rotation(), X_WThrow.translation() +  p_ThrowB_W)
    p_CatchB_W = X_WCatch.rotation() @ p_GB_G
    X_WCatchB_W = RigidTransform(X_WCatch.rotation(), X_WCatch.translation() +  p_CatchB_W)
    meshcat.SetObject("throw", Sphere(squash_ball_radius), rgba=Rgba(.9, .1, .1, 1))
    meshcat.SetTransform("throw", X_WThrowB_W)
    meshcat.SetObject("catch", Sphere(squash_ball_radius), rgba=Rgba(.1, .9, .1, 1))
    meshcat.SetTransform("catch", X_WCatchB_W)

Manually set the throw and catch points

In [ ]:
p_GB_G = [0, 0.11, 0]
def add_target_objects(X_WThrow, X_WCatch):
    p_ThrowB_W = X_WThrow.rotation() @ p_GB_G
    X_WThrowB_W = RigidTransform(X_WThrow.rotation(), X_WThrow.translation() +  p_ThrowB_W)
    p_CatchB_W = X_WCatch.rotation() @ p_GB_G
    X_WCatchB_W = RigidTransform(X_WCatch.rotation(), X_WCatch.translation() +  p_CatchB_W)
    meshcat.SetObject("throw", Sphere(squash_ball_radius), rgba=Rgba(.9, .1, .1, 1))
    meshcat.SetTransform("throw", X_WThrowB_W)
    meshcat.SetObject("catch", Sphere(squash_ball_radius), rgba=Rgba(.1, .9, .1, 1))
    meshcat.SetTransform("catch", X_WCatchB_W)
    return X_WThrowB_W, X_WCatchB_W

q_Throw = np.array([-0.54, 0.58, 0, -1.79, 0, -0.79, 0])
p_WThrow = [0.57, -0.34, 0.39]
R_WThrow = RollPitchYaw([-np.pi, 0.00, 1.03])

q_Catch = np.array([-0.74, 0.58, 0, -1.79, 0, -0.79, 0])
p_WCatch = [0.49, -0.45, 0.39]
R_WCatch = RollPitchYaw([-np.pi, 0.00, 0.83])

X_WThrow = RigidTransform(R_WThrow, p_WThrow)
X_WCatch = RigidTransform(R_WCatch, p_WCatch)

X_WThrowB_W, X_WCatchB_W = add_target_objects(X_WThrow, X_WCatch)
#####



# final joint positions [-0.901 1.465 -0.690 -0.143 -0.908 -1.823 0.001 -0.053 0.054]


p_WCatch_adj = [[0.47, -0.43, 0.35], [0.2, -0.2, 0.2], [0.573, -0.526, 0.375]]

# WSG gripper
opened = np.array([0.035]) #[0.035]
closed = np.array([0.02])

# Differential IK Controller

Use diffIK controller to get other joint configurations around the key points, navigated to using spatial translation

In [ ]:
def get_q_with_diffik(q_original, p_target):
    builder = DiagramBuilder()
    model_directives = """
directives:
- add_model:
    name: iiwa
    file: package://one_arm_juggling/models/iiwa_7_no_collision_modified.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://one_arm_juggling/models/schunk_wsg_50_with_tip_modified_origin.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
    """
    time_step = 0.001
    # def manipulation_station_callback(plant):
    #     plant.AddFrame(
    #         FixedOffsetFrame("extra_frame", plant.GetFrameByName("iiwa_link_7"), RigidTransform(RotationMatrix(), [0.5,0,0]))
    #     )
    #     print("inside prefinalize callback")
    station = builder.AddSystem(
        MyMakeManipulationStation(model_directives=model_directives,
        package_xmls=["./package.xml"], time_step=time_step))
    plant = station.GetSubsystemByName("plant")
    controller_plant = station.GetSubsystemByName(
        "iiwa_controller").get_multibody_plant_for_control()

    # Add a meshcat visualizer.
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat)
    meshcat.ResetRenderMode()
    meshcat.DeleteAddedControls()

    # print(controller_plant.GetFrameByName("extra_frame"))
    # Set up differential inverse kinematics.
    differential_ik = AddIiwaDifferentialIK(
        builder,
        plant,
        frame=controller_plant.GetFrameByName("extra_frame"))

    
    builder.Connect(differential_ik.get_output_port(),
                    station.GetInputPort("iiwa_position"))
    builder.Connect(station.GetOutputPort("iiwa_state_estimated"),
                    differential_ik.GetInputPort("robot_state"))

    # p_I7B_I7 = [0, 0, 0.11 + 0.09]
    # Set up teleop widgets.
    teleop = builder.AddSystem(
        MeshcatPoseSliders(
            meshcat,
            min_range=MeshcatPoseSliders.MinRange(roll=0,
                                                  pitch=-0.5,
                                                  yaw=-np.pi,
                                                  x=-0.6,
                                                  y=-0.8,
                                                  z=0.0),
            max_range=MeshcatPoseSliders.MaxRange(roll=2 * np.pi,
                                                  pitch=np.pi,
                                                  yaw=np.pi,
                                                  x=0.8,
                                                  y=0.3,
                                                  z=1.1),
            body_index=plant.GetBodyByName("iiwa_link_7").index()))
    builder.Connect(teleop.get_output_port(0),
                    differential_ik.get_input_port(0))
    builder.Connect(station.GetOutputPort("body_poses"),
                    teleop.GetInputPort("body_poses"))

    # wsg_teleop = builder.AddSystem(WsgButton(meshcat))
    # builder.Connect(wsg_teleop.get_output_port(0),
    #                 station.GetInputPort("wsg_position"))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()

    target = RigidTransform(RotationMatrix(), p_target)
    meshcat.SetObject("target", Sphere(squash_ball_radius), rgba=Rgba(.3, .1, .1, 1))
    meshcat.SetTransform("target", target)
    
    plant_context = plant.GetMyMutableContextFromRoot(context)
    q0 = plant.GetPositions(plant_context)
    non_iiwa_q0 = q0[7:]
    plant.SetPositions(plant_context, np.concatenate((q_original, non_iiwa_q0)))
    print(f"initial joint positions {plant.GetPositions(plant_context)[0:7]}")
    print(f"planned initial joint position {q_original}")
    simulator.AdvanceTo(0.1)
    while simulator.get_context().get_time() < 10:
        simulator.set_target_realtime_rate(0.1)
        teleop.SetXyz(p_target)
        simulator.AdvanceTo(simulator.get_context().get_time() + 1.0)
        
    print(f"final spatial positions {teleop._get_transform()}")
    print(f"planned final spatial position {p_target}")
    print(f"final joint positions {plant.GetPositions(plant_context)}")
    
    
    return plant.GetPositions(plant_context)[:7]

get_q_with_diffik(q_Catch,p_WCatch_adj[2]) #p_WCatch_adj[0] p_WCatch

SystemExit: Failure at bazel-out/k8-opt/bin/tools/install/libdrake/_virtual_includes/drake_shared_library/drake/manipulation/planner/differential_inverse_kinematics.h:257 in set_joint_velocity_limits(): condition 'v_bounds.first.size() == get_num_velocities()' failed.

/shared-libs/python3.8/py-core/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=68a898be-bd4e-4046-916a-6293aadf6c5d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>